In [1]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['SMB', 'HML', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = 'F-F_Research_Data_Factors.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [3]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = data

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,SMB,HML,Mkt
date,,,
196307,-0.55,-0.81,-0.12
196308,-0.95,1.65,5.32
196309,-0.30,0.19,-1.30
196310,-0.54,-0.09,2.82
196311,-1.13,1.71,-0.58


In [4]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [16]:
# sætter 1/N som vægt
weights = [1 / len(actives)] * len(actives)
weights /= np.sum(weights)
weights

array([0.33333333, 0.33333333, 0.33333333])

In [6]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data)):
    if month + 60 >= len(returns_data):
        break
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month+59]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
    
    # udregner profit, volatilitet og sharpe NB: det risikofrie aktiver er 0% i dette tilfælde
    returns = np.dot(weights, returns_mean)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = returns / volatility

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [7]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [8]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [12]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,SMB Weight,HML Weight,Mkt Weight
0,0.798305,1.770234,0.450960,0.333333,0.333333,0.333333
1,0.815819,1.762329,0.462921,0.333333,0.333333,0.333333
2,0.792768,1.755364,0.451626,0.333333,0.333333,0.333333
3,0.829548,1.751179,0.473708,0.333333,0.333333,0.333333
4,0.859322,1.764341,0.487050,0.333333,0.333333,0.333333


In [13]:
# finder mean-sharpe
df['Sharpe Ratio'].mean()

0.31919943134394635